In [1]:
import nltk
import random

In [2]:
# build dictionary once using corpus
def create_prob_dict(n, corpus, deterministic):
    dict = {}
    for window_begin in range(len(corpus)-n):
        window_end = window_begin+n
        slice = corpus[window_begin:window_end]
        dict_key = tuple(slice[:n-1])
        dict_val_key = slice[n-1]
        if dict_key in dict:
            dict_val = dict[dict_key]
            if dict_val_key in dict_val:
                dict_val[dict_val_key] += 1
            else:
                dict_val[dict_val_key] = 1
        else:
            dict[dict_key] = {dict_val_key: 1}
    return dict

In [3]:
def finish_sentence(sentence, n, corpus, deterministic=False):
    fin_sentence = sentence
    dict = create_prob_dict(n, corpus, deterministic)
    while(True):
        n_gram = tuple(fin_sentence[-(n-1):])
        if n_gram in dict:
            ret_word = find_word(n_gram, dict, deterministic)
            fin_sentence.append(ret_word)
        else:
            # stupid backoff
            curr_n = n
            while(curr_n > 0):
                if(curr_n==1):
                    fin_sentence.append(max(set(corpus), key = corpus.count) )
                    break
                dict.update(create_prob_dict(curr_n, corpus, deterministic))
                curr_n_gram = tuple(fin_sentence[-(curr_n-1):])
                if curr_n_gram in dict:
                    ret_word_n = find_word(curr_n_gram, dict, deterministic)
                    fin_sentence.append(ret_word_n)
                    break
                else:
                    curr_n-=1
                    continue
        if(fin_sentence[-1][0] == '.' or len(fin_sentence)==15):
            break;
    return fin_sentence


In [4]:
def find_word(n_gram, dict, deterministic):
    possible_words = dict[n_gram]
    max_count = 0
    ret_word = ''
    if(deterministic):
        for word in possible_words:
            # choose word with max count
            if(possible_words[word] > max_count):
                max_count = possible_words[word]
                ret_word = word
            elif(possible_words[word] == max_count):
                # tie breaking to choose lesser word
                if(word<ret_word):
                    ret_word = word
    else:
        # choosing from distribution for non-deterministic case
        dict_keys = list(possible_words.keys())
        dict_values = list(possible_words.values())
        ret_word = random.choices(dict_keys, weights=dict_values)[0]
    return ret_word

In [6]:
sentence = ['she', 'was', 'not']
n = 3
corpus = [
    w.lower() for w in
    nltk.corpus.gutenberg.words('austen-sense.txt')
]
result = finish_sentence(sentence, n, corpus, True)
print(result)

['she', 'was', 'not', 'in', 'the', 'world', ',', 'and', 'the', 'two', 'miss', 'steeles', ',', 'as', 'she']
